In [0]:
%sql

-- Qual é o perfil predominante dos animais que chegam ao abrigo em termos de espécie, idade e condição?
-- Respostas: As principais espécies que chegam ao abrigo são cachorros e gatos. Em, sua maioria, cachorros maiores que uma ano. A condição não parece influenciar. 

SELECT
  da.especie,
  f.idade_entrada,
  dc.condicao_entrada,
  COUNT(*) AS qtd_animais
FROM fato_entrada f
JOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
JOIN dim_condicao dc
  ON f.id_dim_condicao = dc.id_dim_condicao
GROUP BY
  da.especie,
  f.idade_entrada,
  dc.condicao_entrada
ORDER BY qtd_animais DESC
LIMIT 10



especie,idade_entrada,condicao_entrada,qtd_animais
Dog,1 year,Normal,8488
Dog,2 years,Normal,7704
Cat,1 month,Normal,4576
Dog,3 years,Normal,3678
Cat,1 year,Normal,2857
Dog,4 years,Normal,2270
Dog,1 month,Normal,2234
Dog,5 years,Normal,1999
Cat,2 years,Normal,1996
Cat,2 months,Normal,1937


In [0]:
%sql
-- Qual é o perfil predominante dos animais que saem do abrigo em termos de espécie, idade, sexo e castração?
-- Resposta: Cães são as principais espécies que chegam aos abrigos. Em sua maioria, jovens (até um ano) e castrados. 

SELECT
  das.especie,
  das.castrado,
  fs.idade_saida,
  dtis.tipo_saida,
  COUNT(*) AS qtd_animais
FROM fato_saida fs
JOIN dim_animal_saida das
  ON fs.id_dim_animal_saida = das.id_dim_animal_saida
JOIN dim_tipo_saida dtis
  ON fs.id_dim_tipo_saida = dtis.id_dim_tipo_saida
GROUP BY
  das.especie,
  das.castrado,
  fs.idade_saida,
  dtis.tipo_saida
ORDER BY qtd_animais DESC
LIMIT 10

especie,castrado,idade_saida,tipo_saida,qtd_animais
Dog,Yes,1 year,Adoption,4684
Cat,Yes,2 months,Adoption,4636
Dog,Yes,2 years,Adoption,3532
Dog,Yes,2 months,Adoption,2453
Dog,Yes,2 years,Return to Owner,1933
Cat,Yes,3 months,Adoption,1716
Dog,Yes,1 year,Return to Owner,1603
Dog,Yes,3 years,Adoption,1483
Dog,Yes,3 years,Return to Owner,1213
Dog,Yes,1 year,Transfer,1109


In [0]:
%sql
-- Qual é a distribuição dos principais tipos de entradas dos animais?
-- Principais são cachorros de ruas e devolvidos pelo o dono. Devolvidos pelo dono é um caso crítico quando pensamos em adoções. Vamos analisar melhor.

SELECT
  dti.tipo_entrada,
  COUNT(*) AS qtd
FROM fato_entrada f
JOIN dim_tipo dti
  ON f.id_dim_tipo = dti.id_dim_tipo
GROUP BY dti.tipo_entrada
order by qtd DESC

tipo_entrada,qtd
Stray,52789
Owner Surrender,14744
Public Assist,4788
Euthanasia Request,232
Wildlife,2


In [0]:
%sql
-- Perfil predominante dos animais devolvidos pelo dono?
-- O perfil predominante dos animais devolvidos são cachorros que já passaram pela fase de filhote (animais maiores que 1 ano)

SELECT
  da.especie,
  f.idade_entrada,
  COUNT(*) AS qtd_animais
FROM fato_entrada f
JOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
JOIN dim_tipo dti
  ON f.id_dim_tipo = dti.id_dim_tipo
WHERE dti.tipo_entrada = 'Owner Surrender'
GROUP BY
  da.especie,
  f.idade_entrada
ORDER BY qtd_animais DESC
LIMIT 10

especie,idade_entrada,qtd_animais
Dog,1 year,1679
Dog,2 years,1374
Dog,3 years,796
Cat,1 year,713
Cat,1 month,634
Cat,2 months,565
Dog,4 years,526
Cat,2 years,507
Dog,5 years,469
Dog,1 month,469


In [0]:
%sql
-- Os animais devolvidos, em sua maioria, estão normais.

SELECT
  dc.condicao_entrada,
  COUNT(*) AS qtd_animais
FROM fato_entrada f
JOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
JOIN dim_condicao dc
  ON f.id_dim_condicao = dc.id_dim_condicao
JOIN dim_tipo dti
  ON f.id_dim_tipo = dti.id_dim_tipo
WHERE dti.tipo_entrada = 'Owner Surrender'
GROUP BY
  dc.condicao_entrada
ORDER BY qtd_animais DESC
LIMIT 10

condicao_entrada,qtd_animais
Normal,13830
Sick,461
Injured,259
Nursing,110
Aged,67
Feral,9
Pregnant,8


In [0]:
%sql
-- Qual é a distribuição dos principais desfechos dos animais (adoção, retorno, transferência, eutanásia)?

SELECT
  dtis.tipo_saida,
  COUNT(*) AS qtd
FROM fato_saida fs
JOIN dim_tipo_saida dtis
  ON fs.id_dim_tipo_saida = dtis.id_dim_tipo_saida
GROUP BY dtis.tipo_saida
order by qtd DESC

tipo_saida,qtd
Adoption,33944
Transfer,20988
Return to Owner,14920
Euthanasia,2803
Died,480
Missing,45


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
 -- Especie

SELECT
  da.especie,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
GROUP BY
  da.especie
ORDER BY tempo_medio_dias DESC


especie,tempo_medio_dias
Livestock,994.0
Dog,546.0
Cat,520.0
Bird,420.0


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
 -- Raca
SELECT
  da.raca,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
AND da.especie = 'Livestock'
GROUP BY
  da.raca
ORDER BY tempo_medio_dias DESC

raca,tempo_medio_dias
Pig Mix,1061.0
Goat Mix,860.0


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
 -- Raca
SELECT
  da.raca,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
AND da.especie = 'Dog'
GROUP BY
  da.raca
ORDER BY tempo_medio_dias DESC
LIMIT 5

raca,tempo_medio_dias
Border Collie/American Staffordshire Terrier,1590.0
Shetland Sheepdog/Italian Greyhound,1569.0
Border Collie/Anatol Shepherd,1562.0
Australian Kelpie/Shiba Inu,1539.0
Plott Hound/Carolina Dog,1515.0


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
 -- Raca
SELECT
  da.raca,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
AND da.especie = 'Cat'
GROUP BY
  da.raca
ORDER BY tempo_medio_dias DESC
LIMIT 5

raca,tempo_medio_dias
Devon Rex,1253.0
Snowshoe/Ragdoll,1053.0
American Wirehair Mix,1042.0
Bengal,995.0
Bengal Mix,944.0


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
 -- Raca
SELECT
  da.raca,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
AND da.especie = 'Bird'
GROUP BY
  da.raca
ORDER BY tempo_medio_dias DESC
LIMIT 5

raca,tempo_medio_dias
Rhode Island Mix,1415.0
Parrot Mix,1159.0
Chicken,957.0
Parakeet Mix,743.0
Silkie Mix,457.0


In [0]:
%sql
-- Como o tempo médio de permanência no abrigo varia de acordo com espécie, raca e idade?
-- Animais muito jovens não podem sair do abrigo pois precisam de cuidados especiais até atingir certa idade para adoção. 
-- Animais mais velhos tedem a ficar mais tempo nos abrigos sem serem adotados. 

SELECT
  f.idade_entrada,
  round(AVG(DATEDIFF(dts.data_saida,dt.data_entrada)),0) AS tempo_medio_dias
FROM fato_entrada f
JOIN fato_saida fs
  ON f.id_dim_animal = fs.id_dim_animal_saida
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
jOIN dim_animal da
  ON f.id_dim_animal = da.id_dim_animal
WHERE dts.data_saida >= dt.data_entrada
GROUP BY
  f.idade_entrada
ORDER BY tempo_medio_dias DESC
LIMIT 10

idade_entrada,tempo_medio_dias
6 days,652.0
22 years,589.0
9 months,587.0
20 years,586.0
14 years,583.0
11 months,567.0
3 years,559.0
7 years,557.0
13 years,557.0
15 years,555.0


In [0]:
%sql

-- Existem padrões sazonais no volume de entradas e adoções ao longo do ano?
--- Entradas
SELECT
  MONTH(dt.data_entrada) AS mes_entrada,
  COUNT(*) AS qtd_animais
FROM fato_entrada f
JOIN dim_tempo dt
  ON f.id_dim_data = dt.id_dim_data
GROUP BY
  MONTH(dt.data_entrada)
ORDER BY
  qtd_animais DESC
LIMIT 3;

mes_entrada,qtd_animais
5,7141
10,6991
6,6701


In [0]:
%sql
-- Existem padrões sazonais no volume de entradas e adoções ao longo do ano?
--- Saidas: Datas especiais (dia das crianças, ferias e natal)
SELECT
  MONTH(dts.data_saida) AS mes_saida,
  COUNT(*) AS qtd_animais
FROM fato_saida fs
JOIN dim_tempo_saida dts
  ON fs.id_dim_data_saida = dts.id_dim_data_saida
GROUP BY
  MONTH(dts.data_saida)
ORDER BY
  qtd_animais DESC
LIMIT 3;

mes_saida,qtd_animais
10,7107
7,6835
12,6630
